In [1]:
!pip install autokeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.8/466.8 KB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

PATH = "/content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/"

train = pd.read_csv(PATH+'data/train.csv')
test = pd.read_csv(PATH+'data/test.csv')
submission = pd.read_csv(PATH+'data/sample_submission.csv')

Mounted at /content/drive


## 1. 전처리

In [3]:
train["text"]=train["prompt"]+train["answer"]
train=train[["text","AI"]]

In [4]:
test["text"]=test["prompt"]+test["answer"]
test=test["text"]

In [5]:
train["text"].isna().sum()

1

In [6]:
train = train[train["text"].isna()==False]

In [7]:
train["AI"].isna().sum()

1

In [8]:
train = train[train["AI"].isna()==False]

In [9]:
test.isna().sum()

0

In [10]:
train

,text,AI
0,What is the future of AI?The future of Artific...,1.0
1,What is your biggest challenge in your career?...,0.0
2,What is the tallest mountain in the world?The ...,1.0
3,What are the best 5 super cars?McLaren 720S.\r...,0.0
4,What is deep learning?a type of machine learni...,0.0
...,...,...
743,What is the most important thing in a family?T...,0.0
744,What are the best machine learning Models?Line...,0.0
745,How can I chop onions without crying?The best ...,0.0
746,What is the best way to stay healthy?The best ...,0.0


In [11]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=c97eb5ed86b88a6ad5945cf3cbeca702bb02908ee2aa5d7eb438bdec9559a6b2
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [12]:
import numpy as np

In [13]:
import re
def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)
    
import emoji
def dataPreprocessing(x): 
    x = x.lower()
    x = removeHTML(x)
    x = emoji.demojize(x, delimiters=(" ", " "))
    x = re.sub("@\w+", '',x) # removing mentions (@)
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    x = re.sub(r"[^\w\s]", '',x) # to remove symbols
    x = re.sub("http\w+", '',x)
    x = re.sub("\s[a-z]\s", '',x)
    x = x.strip()
    return x

In [14]:

train_text=train["text"].apply(lambda x: dataPreprocessing(x))

In [15]:
test_text=test.apply(lambda x: dataPreprocessing(x))

In [16]:
train_text

0      what is the future of aithe future of artifici...
1      what is your biggest challenge in your careerm...
2      what is the tallest mountain in the worldthe t...
3      what are the best  super carsmclaren\nferrari ...
4      what is deep learninga type of machine learnin...
                             ...                        
743    what is the most important thing infamilythe m...
744    what are the best machine learning modelslinea...
745    how canchop onions without cryingthe best thin...
746    what is the best way to stay healthythe best w...
747    do you think naruto is overratedi think naruto...
Name: text, Length: 746, dtype: object

In [17]:
from sklearn.model_selection import train_test_split

X = train_text
y = train['AI']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
pd.DataFrame(X_train, columns=['text'])

,text
70,what is the name of the unit used to measure p...
164,what is the name of the largest underwater cav...
712,what is the name of the gas that makes up the ...
265,what is the name of the largest underwater cav...
250,what is the most important factor in makingdec...
...,...
71,what is your biggest strengthmy biggest streng...
106,what is the currency of australiaaustralian do...
270,what is the best way to stay positivethe best ...
435,what is the name of the largest underwater cav...


In [19]:
pd.DataFrame(y_train, columns=['AI'])

,AI
70,1.0
164,0.0
712,0.0
265,0.0
250,0.0
...,...
71,0.0
106,0.0
270,0.0
435,0.0


In [20]:
# train_data = pd.concat([pd.DataFrame(X_train, columns=['text']), pd.DataFrame(y_train, columns=['AI'])], axis=1)
# train_data

In [21]:
# test_data = pd.concat([pd.DataFrame(X_test, columns=['text']), pd.DataFrame(y_test, columns=['AI'])], axis=1)
# test_data

# 2. 모델링

In [37]:
import os

import numpy as np
import tensorflow as tf
from sklearn.datasets import load_files

import autokeras as ak

In [51]:
# Initialize the text classifier.
clf = ak.TextClassifier(
    overwrite=True, max_trials=10, objective='val_accuracy', seed=32
)  

In [52]:
# 모델 저장 경로
model_dir = PATH+"model/autokeras_trial10_230304/"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

# 체크포인트 파일 저장 경로
checkpoint_path = model_dir+"checkpoints_model_autokeras_230304.ckpt"

In [53]:
callbacks = [
tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, save_weights_only=True, verbose=1, save_best_only=True
),
tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_accuracy", factor=0.5, patience=5, min_lr=0.0001
),
tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode='max', patience=10, verbose=1),
]

In [54]:
# Feed the text classifier with training data.
clf.fit(X_train.values, y_train.values, callbacks=callbacks, validation_split=0.2, epochs=30)

Trial 10 Complete [00h 00m 01s]

Best val_accuracy So Far: 0.9285714030265808
Total elapsed time: 00h 27m 36s
Epoch 1/30
19/19 [==============================] - ETA: 0s - loss: 0.6181 - accuracy: 0.7282

19/19 [==============================] - 56s 365ms/step - loss: 0.6181 - accuracy: 0.7282 - lr: 6.5455e-06
Epoch 2/30
19/19 [==============================] - ETA: 0s - loss: 0.4922 - accuracy: 0.7668

19/19 [==============================] - 5s 280ms/step - loss: 0.4922 - accuracy: 0.7668 - lr: 1.3455e-05
Epoch 3/30
19/19 [==============================] - ETA: 0s - loss: 0.3488 - accuracy: 0.8658

19/19 [==============================] - 5s 276ms/step - loss: 0.3488 - accuracy: 0.8658 - lr: 1.7926e-05
Epoch 4/30
19/19 [==============================] - ETA: 0s - loss: 0.2523 - accuracy: 0.9044

19/19 [==============================] - 5s 279ms/step - loss: 0.2523 - accuracy: 0.9044 - lr: 1.7222e-05
Epoch 5/30
19/19 [==============================] - ETA: 0s - loss: 0.1717 - accuracy: 0.9262

...

19/19 [==============================] - 5s 281ms/step - loss: 0.0171 - accuracy: 0.9933 - lr: 1.7407e-06
Epoch 27/30
19/19 [==============================] - ETA: 0s - loss: 0.0229 - accuracy: 0.9866

19/19 [==============================] - 5s 280ms/step - loss: 0.0229 - accuracy: 0.9866 - lr: 1.0370e-06
Epoch 28/30
19/19 [==============================] - ETA: 0s - loss: 0.0202 - accuracy: 0.9849

19/19 [==============================] - 5s 280ms/step - loss: 0.0202 - accuracy: 0.9849 - lr: 3.3333e-07
Epoch 29/30
19/19 [==============================] - ETA: 0s - loss: 0.0202 - accuracy: 0.9883

19/19 [==============================] - 5s 278ms/step - loss: 0.0202 - accuracy: 0.9883 - lr: 0.0000e+00
Epoch 30/30
19/19 [==============================] - ETA: 0s - loss: 0.0182 - accuracy: 0.9899

19/19 [==============================] - 5s 282ms/step - loss: 0.0182 - accuracy: 0.9899 - lr: 0.0000e+00


# 3. 성능 평가

In [56]:
from sklearn.metrics import classification_report

In [58]:
# Predict with the best model.
predicted_y = clf.predict(X_test.values)
# Evaluate the best model with testing data.
print(classification_report(y_test.values, predicted_y))

5/5 [==============================] - 1s 194ms/step
              precision    recall  f1-score   support

         0.0       0.96      0.87      0.91       104
         1.0       0.75      0.91      0.82        46

    accuracy                           0.88       150
   macro avg       0.85      0.89      0.87       150
weighted avg       0.89      0.88      0.88       150



In [59]:
# model_to_use = predictor.get_model_best()
model_pred = clf.predict(test_text.values)

8/8 [==============================] - 2s 191ms/step


In [60]:
submission['Category'] = model_pred

In [61]:
submission = submission.astype({'Category':'int64'})

In [62]:
submission

,Id,Category
0,710,1
1,487,0
2,136,1
3,44,1
4,627,0
...,...,...
244,702,1
245,500,1
246,818,0
247,584,0


In [63]:
submission.to_csv(PATH+'submit/autokeras_text_callback_submission_230304.csv', index=False)